In [1]:
import sys
import pandas as pd
import numpy as np
import time
from datetime import datetime
#from apscheduler.scheduler import Scheduler

import io
import requests

import elex as elex
from elex.api import Election

print("ready")

ready


In [3]:
#You need an account with AP to use their API
# NOTE this is set up for tests, not live
election = Election(electiondate='2016-03-15', testresults=False, liveresults=True, is_test=False)
df=pd.read_csv('ILcands.csv', dtype=object)

# this is the list of results using serialize
races = pd.DataFrame([f.serialize() for f in election.results])
races.to_csv('csv/races' + str( datetime.now() ) + '.csv', index=False, encoding="utf-8")

allraces = races[['raceid','unique_id','first', 'last', 'officename', 'party', 'reportingunitid', 'statename' , 'reportingunitname', 'precinctsreportingpct', 'precinctsreporting', 'precinctstotal', 'votecount', 'votepct','winner']]
racecols3 = pd.DataFrame( allraces.loc[ (allraces["statename"] == "Illinois" ) ] )
racecols2 = pd.DataFrame( racecols3.loc[ (racecols3["officename"].isin([ 'President','U.S. Senate' ]) ) ] )
#racecols = pd.DataFrame( racecols1.loc[ ( racecols1['unique_id'].isin(['polid-1746','polid-1445','polid-8639','polid-61815','polid-53044','polid-36679' ]) ) ] )
racecols = pd.DataFrame( racecols2.loc[ (racecols2["reportingunitid"].isin([ 'state-1','county-14016','county-14022','county-14045','county-14049','county-14056','county-14099' ]) ) ] )

# First -- Cook all
df_cook = pd.DataFrame( racecols.loc[ (racecols["reportingunitid"] == "county-14016" ) ] )
df_cook['totalvotes'] = df_cook['votecount'].groupby(df_cook['raceid']).transform('sum')
df_cook = df_cook[['unique_id', 'precinctsreporting','votecount','precinctstotal', 'totalvotes' ]]
left = df
right = df_cook
df = pd.merge(left, right, how='left', on='unique_id', suffixes=('', '_cook'))
# Second -- DuPage
df_dupage = pd.DataFrame( racecols.loc[ (racecols["reportingunitid"] == "county-14022" ) ] )
df_dupage['totalvotes'] = df_dupage['votecount'].groupby(df_dupage['raceid']).transform('sum')
df_dupage = df_dupage[['unique_id', 'precinctsreportingpct', 'votecount', 'votepct', 'totalvotes' ]]
left = df
right = df_dupage
df = pd.merge(left, right, how='left', on='unique_id', suffixes=('', '_dupage'))
# Third -- Kane
df_kane = pd.DataFrame( racecols.loc[ (racecols["reportingunitid"] == "county-14045" ) ] )
df_kane['totalvotes'] = df_kane['votecount'].groupby(df_kane['raceid']).transform('sum')
df_kane = df_kane[['unique_id', 'precinctsreportingpct', 'votecount', 'votepct', 'totalvotes' ]]
left = df
right = df_kane
df = pd.merge(left, right, how='left', on='unique_id', suffixes=('', '_kane'))
# Fourth -- Lake
df_lake = pd.DataFrame( racecols.loc[ (racecols["reportingunitid"] == "county-14049" ) ] )
df_lake['totalvotes'] = df_lake['votecount'].groupby(df_lake['raceid']).transform('sum')
df_lake = df_lake[['unique_id', 'precinctsreportingpct', 'votecount', 'votepct', 'totalvotes' ]]
left = df
right = df_lake
df = pd.merge(left, right, how='left', on='unique_id', suffixes=('', '_lake'))
# Fifth -- McHenry
df_mch = pd.DataFrame( racecols.loc[ (racecols["reportingunitid"] == "county-14056" ) ] )
df_mch['totalvotes'] = df_mch['votecount'].groupby(df_mch['raceid']).transform('sum')
df_mch = df_mch[['unique_id', 'precinctsreportingpct', 'votecount', 'votepct', 'totalvotes' ]]
left = df
right = df_mch
df = pd.merge(left, right, how='left', on='unique_id', suffixes=('', '_mch'))
# Sixth -- Will
df_will = pd.DataFrame( racecols.loc[ (racecols["reportingunitid"] == "county-14099" ) ] )
df_will['totalvotes'] = df_will['votecount'].groupby(df_will['raceid']).transform('sum')
df_will = df_mch[['unique_id', 'precinctsreportingpct', 'votecount', 'votepct', 'totalvotes' ]]
left = df
right = df_will
df = pd.merge(left, right, how='left', on='unique_id', suffixes=('', '_will'))

# ___________________________________________________________
# Grabbing the Chicago data
#Reading info from a url-basedtext file
url="http://www.chicagoelections.com/results/ap/summary.txt"
#this is for when it goes to that string only
#s=requests.get(url).content
#df_chi=pd.read_csv(io.StringIO(s.decode('utf-8')))
df_chi=pd.read_csv('SummaryExport.txt')

# Parsing the file
df_chi.columns = ['a']
df_chi['race'] = df_chi.a.str[:4]
df_chi['chiID'] = df_chi.a.str[:7]
df_chi['votecount'] = df_chi.a.str[11:18]
df_chi['prec_total'] = df_chi.a.str[7:11]
df_chi['prec_in'] = df_chi.a.str[18:22]

# Working the data
df_chi['votecount'] = df_chi["votecount"].astype(float)
df_chi['precinctsreportingpct'] = df_chi["prec_in"].astype(float) / df_chi["prec_total"].astype(float)
#df_chi['precinctsreportingpct'] = pd.Series([round(val * 100, 1) for val in df_chi['precinctsreportingpct']], index = df_chi.index)

# Groups by race and sums up all votes in race, assigns to all votes
df_chi['totalvotes'] = df_chi['votecount'].groupby(df_chi['race']).transform('sum')
#allvotes = df['votes'].groupby(df['race']).transform('sum')
df_chi['votepct'] = df_chi["votecount"] / df_chi["totalvotes"]
#df_chi['votepct'] = pd.Series([round(val * 100, 1) for val in df_chi['votepct']], index = df_chi.index)

chiDF = df_chi[['chiID', 'precinctsreportingpct', 'votecount', 'votepct', 'totalvotes' ]]
chiDF.to_csv('csv/chiDF' + str( datetime.now() ) + '.csv', index=False, encoding="utf-8")

# Joins chi data
left = df
right = chiDF
df = pd.merge(left, right, how='left', on='chiID', suffixes=('', '_chi'))

# ___________________________________________________________

# Last -- all Illinois
df_statewide = pd.DataFrame( racecols.loc[ (racecols["reportingunitid"] == "state-1" ) ] )
df_statewide['totalvotes'] = df_statewide['votecount'].groupby(df_statewide['raceid']).transform('sum')
df_statewide = df_statewide[['unique_id', 'precinctsreportingpct', 'votecount', 'votepct', 'totalvotes' ]]
left = df
right = df_statewide
df = pd.merge(left, right, how='left', on='unique_id', suffixes=('', '_statewide'))

df.to_csv('csv/ILstate' + str( datetime.now() ) + '.csv', index=False, encoding="utf-8")
df.head(3)

,dhID,chiID,unique_id,CandName,party,precinctsreporting,votecount,precinctstotal,totalvotes,precinctsreportingpct,...,votepct_will,totalvotes_will,precinctsreportingpct_chi,votecount_chi,votepct_chi,totalvotes_chi,precinctsreportingpct_statewide,votecount_statewide,votepct_statewide,totalvotes_statewide
0,1,0010001,polid-1746,Hillary Clinton,Dem,0,0,3668,0,0,...,0,0,1,989,0.189827,5210,0,0,0,0
1,1,0010006,polid-1445,Bernie Sanders,Dem,0,0,3668,0,0,...,0,0,1,802,0.153935,5210,0,0,0,0
2,2,0163003,polid-8639,Donald Trump,GOP,0,0,3668,0,0,...,0,0,1,867,0.263046,3296,0,0,0,0
